In [322]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
label_encoder = LabelEncoder()

In [323]:
df = pd.read_csv('/home/sxluong/151A/CSE151A_MushroomAnalysis/primary_data.csv',sep=';', on_bad_lines='skip')


In [324]:
# Keep naming consistent
df = df.rename(columns={"Cap-surface": "cap-surface"})

In [325]:
# Because of multicolinearity and name has no correlation
df = df.drop(columns=['Spore-print-color',
                     'has-ring','name'])

#Binary Encoding
binary_mapping = {'[t]': 1, '[f]': 0}
df['does-bruise-or-bleed'] = df['does-bruise-or-bleed'].map(binary_mapping)
binary_mapping = {'p': 1, 'e': 0}
df['class'] = df['class'].map(binary_mapping)
                                 

In [326]:
# Too many Nan values where imputation is not good to do
df = df.drop(columns=['stem-root','veil-type','veil-color','stem-surface'])

In [327]:
#Only small subset of families therefore used a label encoder
df['family'] = label_encoder.fit_transform(df['family'])

In [328]:
df['gill-attachment'].fillna('[?]', inplace=True)
df['ring-type'].fillna('[?]', inplace=True)
df['cap-surface'].fillna('[?]', inplace=True)
df['gill-spacing'].fillna('[?]', inplace=True)
# As per the ? key used, we simply filled in the Nans as they are essentially unknown

In [329]:
def convert_dimensions(l):
    
    if len(l) ==2 :
        
        return (float(l[0])+float(l[1]))/2.0
    return float(l[0])

df['cap-diameter'] = df['cap-diameter'].apply(lambda x: convert_dimensions(x.strip('[]').split(', ')))
df['stem-height'] = df['stem-height'].apply(lambda x: convert_dimensions(x.strip('[]').split(', ')))
df['stem-width'] = df['stem-width'].apply(lambda x: convert_dimensions(x.strip('[]').split(', ')))

In [330]:
columns_to_apply_multilabeling = ['cap-shape','cap-surface','cap-color','gill-attachment', 'gill-spacing', 'gill-color','stem-color','ring-type','season']

for col in columns_to_apply_multilabeling:
    mlb = MultiLabelBinarizer()
    column = df[col].apply(lambda x: x.strip('[]').split(', '))
    
    encoded_data = mlb.fit_transform(column)
    encoded_df = pd.DataFrame(encoded_data, columns=mlb.classes_+"_"+col)
    df = df.drop(col, axis=1)
    df = pd.concat([df, encoded_df], axis=1)
